In [1]:
"""
In this notebook we will create the fuel load and last fire % and export the, in a tiff with a 0.1º resolution.
"""

'\nIn this notebook we will create the fuel load and last fire % and export the, in a tiff with a 0.1º resolution.\n'

In [2]:
from pathlib import Path
import geopandas as gpd
import rasterio
import warnings
import numpy as np
import pandas as pd
from rasterstats import zonal_stats
import sys

In [3]:
warnings.simplefilter("ignore", UserWarning)

In [4]:
last_fire_paths = list(Path(r"/Volumes/Externo/PT-FireSprd/").rglob("*_p.tif"))

fuel_paths = list(Path(r"../../Data/Interim/GIS_data/fuel_models/100m/portugal/PT-FireSprd").rglob("*.tif"))

In [5]:
# Associar cada ao ao respetivo years_since_last_fire map and fuel model map

fire_maps = {p.stem: p for p in last_fire_paths}
fuel_maps = {p.parent.stem: p for p in fuel_paths}

print(fire_maps)
print(fuel_maps)

{'years_since_fire_2015_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2015_p.tif'), 'years_since_fire_2015_utm29n_tmp_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2015_utm29n_tmp_p.tif'), 'years_since_fire_2016_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2016_p.tif'), 'years_since_fire_2017_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2017_p.tif'), 'years_since_fire_2018_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2018_p.tif'), 'years_since_fire_2019_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2019_p.tif'), 'years_since_fire_2020_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2020_p.tif'), 'years_since_fire_2021_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2021_p.tif'), 'years_since_fire_2022_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2022_p.tif'), 'years_since_fire_2023_p': PosixPath('/Volumes/Externo/PT-FireSprd/years_since_fire_2023_p.tif'

In [6]:
gdf = gpd.read_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_valid.shp")
gdf = gdf.to_crs("epsg:32629")

# gdf = gpd.read_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_p_meteo.shp")
print(gdf)

      fid             fname  year    id type             sdate  \
0       1  Gouveia_10082015  2015     1    z  2015-08-10 14:30   
1       1  Gouveia_10082015  2015     2    z  2015-08-11 12:30   
2       1  Gouveia_10082015  2015     3    z  2015-08-11 12:30   
3       1  Gouveia_10082015  2015     4    p  2015-08-10 14:30   
4       1  Gouveia_10082015  2015     5    p  2015-08-11 03:00   
...   ...               ...   ...   ...  ...               ...   
3350  164  Sabugal_16082025  2025  3363    p  2025-08-18 13:15   
3351  164  Sabugal_16082025  2025  3364    p  2025-08-19 03:30   
3352  164  Sabugal_16082025  2025  3365    p  2025-08-19 10:30   
3353  164  Sabugal_16082025  2025  3366    p                na   
3354  164  Sabugal_16082025  2025  3367    p                na   

                 edate      inidoy      enddoy source  ...       ros_i  \
0                   na   -1.000000   -1.000000  fserv  ...   -1.000000   
1                   na   -1.000000   -1.000000    sat  ... 

In [7]:
# Função para calcular percentagem de área que ardeu recentemente a -2 anos, -4 -6 e -8

def calc_fire_percentages(geom, year, fire_maps):

    results = {
        "1_3y_fir_p": 0.0,
        "3_8y_fir_p": 0.0, 
        "8_ny_fir_p": 0.0
    }

    key = f"years_since_fire_{year}_p"
    if key not in fire_maps:
        return results  # sem mapa → 0%

    fire_stats = zonal_stats(geom, fire_maps[key], stats=None, nodata=None, raster_out=True, all_touched=False)
    fire_arr = fire_stats[0]["mini_raster_array"]
    mask = fire_stats[0].get("mini_raster_mask")

    if hasattr(fire_arr, "filled"):
        fire_arr = fire_arr.filled(np.nan)
    fire_arr = fire_arr.astype(float)

    if mask is not None:
        fire_arr[mask] = np.nan

    # Pixeis válidos (vegetados): 0 ou mais
    valid = ~np.isnan(fire_arr)
    valid_values = fire_arr[valid]

    if valid_values.size == 0:
        return results

    total = valid_values.size
    fire_positive = valid_values[valid_values > 0]

    if total > 0:
        # 1-3 anos anteriores (inclui 1, 2 e 3 anos)
        results["1_3y_fir_p"] = ((fire_positive >= 1) & (fire_positive <= 3)).sum() / total * 100
        
        # 3-8 anos anteriores (inclui >3 até 8 anos)
        results["3_8y_fir_p"] = ((fire_positive > 3) & (fire_positive <= 8)).sum() / total * 100
        
        # +8 anos anteriores (mais de 8 anos)
        results["8_ny_fir_p"] = (fire_positive > 8).sum() / total * 100

    # PRINT DOS VALORES NÃO NULOS 1-3 e 3-8
    vals_1_3 = fire_positive[(fire_positive >= 1) & (fire_positive <= 3)]
    vals_3_8 = fire_positive[(fire_positive > 3) & (fire_positive <= 8)]
    
    if results["1_3y_fir_p"] > 0:
        print(f"1_3y_fir_p: {results['1_3y_fir_p']}")
    if results["3_8y_fir_p"] > 0:
        print(f"3_8y_fir_p: {results['3_8y_fir_p']}")


    return results

In [8]:
FUEL_LOAD_TABLE = {
    4:   34.67,   # Mato Alto com continuidade h e v
    98:  0.00,    # sem combustível
    221: 15.49,   # M-CAD
    222: 15.04,   # M-ESC
    223: 16.69,   # M-EUC
    227: 17.10,   # M-PIN
    231:  3.55,   # V-Ha
    232:  1.50,   # V-Hb
    233: 26.50,   # V-MAa
    234: 14.00,   # V-MAb
    235:  9.00,   # V-MH
    236: 23.00,   # V-MMa
    237: 11.50    # V-MMb
}


def calc_mean_fuelload(geom, year, fuel_maps, fuel_load_table=FUEL_LOAD_TABLE):

    fuel_path = fuel_maps.get(str(year))
    if not fuel_path:
        return np.nan

    with rasterio.open(fuel_path) as src:
        # --- Tenta primeiro apenas pixels totalmente dentro ---
        stats_full = zonal_stats(
            geom,
            fuel_path,
            categorical=True,
            all_touched=False,  # só pixels totalmente dentro
            nodata=src.nodata
        )

        if not stats_full or not stats_full[0]:
            # Se não houver pixels totalmente dentro, permite parcialmente dentro
            stats_full = zonal_stats(
                geom,
                fuel_path,
                categorical=True,
                all_touched=True,  # inclui parcialmente dentro
                nodata=src.nodata
            )

    counts = stats_full[0]
    if not counts:
        return np.nan

    # Calcula média ponderada (somatório(fuel_load * n_pixels) / total_pixels)
    total_pixels = sum(counts.values())
    if total_pixels == 0:
        return np.nan

    weighted_sum = 0.0
    for fm, n in counts.items():
        fuel_load = fuel_load_table.get(fm)
        if fuel_load is not None:
            weighted_sum += fuel_load * n

    mean_fuelload = weighted_sum / total_pixels if total_pixels > 0 else np.nan
    return mean_fuelload

In [9]:
# Loop principal com uso das funções anteriores

gdf["1_3y_fir_p"] = -1.0
gdf["3_8y_fir_p"] = -1.0
gdf["8_ny_fir_p"] = -1.0

for idx, row in gdf.iterrows():
    geom = row.geometry

    # --- Percentagem de area ardida ---
    fire_perc = calc_fire_percentages(geom, row["year"], fire_maps)
    for col, val in fire_perc.items():
        gdf.at[idx, col] = val

    # --- Fuel Load Médio ---
    gdf.at[idx, "f_load_av"] = calc_mean_fuelload(geom, row["year"], fuel_maps)


3_8y_fir_p: 77.77777777777779
1_3y_fir_p: 5.9429015342785005
3_8y_fir_p: 12.546125461254611
3_8y_fir_p: 26.691729323308273
3_8y_fir_p: 3.90625
3_8y_fir_p: 3.061224489795918
3_8y_fir_p: 15.85144927536232
3_8y_fir_p: 9.322033898305085
1_3y_fir_p: 6.8181818181818175
3_8y_fir_p: 65.9090909090909
1_3y_fir_p: 6.943303204601479
3_8y_fir_p: 35.866885784716516
1_3y_fir_p: 2.7246376811594204
3_8y_fir_p: 26.956521739130434
3_8y_fir_p: 5.399431638774866
3_8y_fir_p: 3.1485284052019162
1_3y_fir_p: 0.8266993263931415
3_8y_fir_p: 11.420698101653398
1_3y_fir_p: 4.284591194968553
1_3y_fir_p: 0.5448106782892944
3_8y_fir_p: 0.490329610460365
1_3y_fir_p: 5.87515299877601
3_8y_fir_p: 18.727050183598532
1_3y_fir_p: 1.0416666666666665
3_8y_fir_p: 2.258469259723965
3_8y_fir_p: 68.18181818181817
1_3y_fir_p: 10.619469026548673
3_8y_fir_p: 78.58472998137802
3_8y_fir_p: 100.0
3_8y_fir_p: 0.5199306759098787
3_8y_fir_p: 1.477832512315271
1_3y_fir_p: 1.2362637362637363
3_8y_fir_p: 5.319148936170213
1_3y_fir_p: 11.988

In [10]:
# Save the GeoDataFrame with the calculated variables
gdf.to_file("fire_fuel_variables.shp")

In [12]:
# Export variables to TIFF with 0.1° resolution
from rasterio.transform import from_bounds
from rasterio.features import rasterize

# Convert GeoDataFrame to WGS84 (EPSG:4326) for 0.1° resolution
gdf_wgs84 = gdf.to_crs("EPSG:4326")

# Get bounds in WGS84
bounds = gdf_wgs84.total_bounds  # [minx, miny, maxx, maxy]
minx, miny, maxx, maxy = bounds

# Define 0.1° resolution
resolution = 0.1

# Calculate raster dimensions
width = int(np.ceil((maxx - minx) / resolution))
height = int(np.ceil((maxy - miny) / resolution))

# Create transform
transform = from_bounds(minx, miny, maxx, maxy, width, height)

# Get unique years in the dataset
years = sorted(gdf_wgs84['year'].unique())
print(f"Available years: {years}")

# Export fuel load for each year (2015-2025)
for year in range(2015, 2026):
    print(f"\nExporting fuel_load for year {year}...")
    
    # Filter data for this year
    gdf_year = gdf_wgs84[gdf_wgs84['year'] == year]
    
    if len(gdf_year) == 0:
        print(f"  No data for year {year}, skipping...")
        continue
    
    # Create list of (geometry, value) tuples
    shapes = [(geom, value) for geom, value in zip(gdf_year.geometry, gdf_year['f_load_av']) if not np.isnan(value)]
    
    if len(shapes) == 0:
        print(f"  No valid fuel load data for year {year}, skipping...")
        continue
    
    # Rasterize
    raster = rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=-9999,  # NoData value
        dtype=rasterio.float32
    )
    
    # Write to file
    output_file = f'fuel_load_{year}.tif'
    with rasterio.open(
        output_file,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=rasterio.float32,
        crs='EPSG:4326',
        transform=transform,
        nodata=-9999,
        compress='lzw'
    ) as dst:
        dst.write(raster, 1)
    
    print(f"  Exported {output_file} with {len(shapes)} geometries")

# Export 8_ny_fir_p and 3_8y_fir_p (aggregate across all years)
print("\n\nExporting fire percentage variables (aggregated across years)...")

fire_variables = {
    '8_ny_fir_p': '8_ny_fir_p.tif',
    '3_8y_fir_p': '3_8y_fir_p.tif'
}

for var_name, output_file in fire_variables.items():
    print(f"\nExporting {var_name} to {output_file}...")
    
    # Create list of (geometry, value) tuples
    shapes = [(geom, value) for geom, value in zip(gdf_wgs84.geometry, gdf_wgs84[var_name]) if not np.isnan(value) and value > 0]
    
    if len(shapes) == 0:
        print(f"  No valid data for {var_name}, skipping...")
        continue
    
    # Rasterize
    raster = rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=-9999,  # NoData value
        dtype=rasterio.float32
    )
    
    # Write to file
    with rasterio.open(
        output_file,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=rasterio.float32,
        crs='EPSG:4326',
        transform=transform,
        nodata=-9999,
        compress='lzw'
    ) as dst:
        dst.write(raster, 1)
    
    print(f"  Exported {output_file} with {len(shapes)} geometries")

print("\n\nAll variables exported successfully!")

Available years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

Exporting fuel_load for year 2015...
  Exported fuel_load_2015.tif with 20 geometries

Exporting fuel_load for year 2016...
  Exported fuel_load_2016.tif with 262 geometries

Exporting fuel_load for year 2017...
  Exported fuel_load_2017.tif with 1358 geometries

Exporting fuel_load for year 2018...
  Exported fuel_load_2018.tif with 88 geometries

Exporting fuel_load for year 2019...
  Exported fuel_load_2019.tif with 184 geometries

Exporting fuel_load for year 2020...
  Exported fuel_load_2020.tif with 233 geometries

Exporting fuel_load for year 2021...
  Exported fuel_load_2021.tif with 70 geometries

Exporting fuel_load for year 2022...
  Exported fuel_load_2022.tif with 245 geometries

Exporting fuel_load for year 2023...
  Exported fuel_load_2023.tif with 204 geometries

Exporting fuel_